<div style="color:white;display:fill;border-radius:8px;
            background-color:#323232;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:white;"><b>ABOUT</b></p>
</div>

This notebook is influenced by https://www.kaggle.com/code/paulorzp/mean-model-jpx

The idea in that notebook is to produce a rank based on the average Target value for the last 34 days

This is a sensible baseline but in the original notebook the implementation is somewhat flawed:

- a static snapshot is taken for computing the mean target
- the snapshot is leaky - it overlaps with the test data
- the target value as provided in the test data is taken - but, afaik, during the evaluation period no target values will be provided

The goal of this notebook is to eliminate the above flaws. The first two flaws are eliminated by taking a "walking snapshot". It is not leaky as we make sure to not include any future values. Once the leak was eliminated the score dropped from 0.405 to 0.174.

Not sure why the author chose 34 days as a suitable interval for computing the mean target value - but I'll stick to it

In [ ]:
import numpy as np
import pandas as pd
import jpx_tokyo_market_prediction
from datetime import timedelta

In [ ]:
NDAYS = 34
prices = pd.read_csv('../input/jpx-tokyo-stock-exchange-prediction/train_files/stock_prices.csv').tail(NDAYS*2000)
prices.head()

In [ ]:
sprices = pd.read_csv("../input/jpx-tokyo-stock-exchange-prediction/supplemental_files/stock_prices.csv")
prices = pd.concat([prices, sprices])
prices.Date = pd.to_datetime(prices.Date)

f = lambda df: df['AdjustmentFactor'][::-1].cumprod()[::-1]

def generate_target(df):
    df.loc[:, "TargetAdjusted"] = df["Close"].pct_change().shift(-2)
    return df

In [ ]:
env = jpx_tokyo_market_prediction.make_env()
iter_test = env.iter_test()

for (nprices, options, financials, trades, secondary_prices, sample_prediction) in iter_test:
    sample_prediction.Date = pd.to_datetime(sample_prediction.Date)
    min_date = min(sample_prediction['Date']) #there should be just one date
    lastdays = prices[(prices['Date'] > min_date - timedelta(days=NDAYS)) & (prices['Date'] < min_date - timedelta(days=2))].reset_index(drop=True)
    lastdays['CumAdjustmentFactor'] = lastdays.groupby('SecuritiesCode').apply(f).values
    lastdays['Close'] *= lastdays['CumAdjustmentFactor']
    lastdays = lastdays.groupby("SecuritiesCode").apply(generate_target)[["SecuritiesCode", "Date", "TargetAdjusted"]]
    lastdays['avg_target'] = lastdays.groupby("SecuritiesCode")['TargetAdjusted'].transform('mean')
    min_date = min(lastdays['Date'])
    lastdays = lastdays[lastdays['Date'] == min_date]
    lastdays = lastdays.sort_values(by = "avg_target", ascending=False)
    lastdays['Rank'] = np.arange(0,2000)
    pred = dict(zip(lastdays["SecuritiesCode"],lastdays["Rank"]))
    sample_prediction['Rank'] = sample_prediction["SecuritiesCode"].map(pred)
    env.predict(sample_prediction)